# Index
Description of this directory.

May 2022 collaboration on the glioma dataset.
We classified images at the patch level following Hou et al 2016.
We used a CNN+Attention as well as CellProfiler+RandomForest.
We subsequently discovered some data problems that force a restart.

1. CellProfiler was tuned and run on the test set.  
1. At least 5 WSI were missing from the 83K test set.  
1. We had used 40X images whereas Hou et al used 20X.  

## Attention 1-5
We analyzed cancer classes 1-5. We did not have data for class 0.
Our CellProfiler pipeline was called Process100.
We ran Process100 on the 83K test set.
This generated nucleus bounding boxes, among other stats.
Attention heatmaps were generated running the CNN on the 83K test set.
Attention was expressed as a percentage per pixel ("temperature").
We extracted the average temperature per nucleus bounding box.
We put temperature in five equal-probabilty bins.
Tackling each cancer class separately,
we trained a RF to predict the temperature per nucleus
given only the CP features per nucleus.
Accuracy was about 94% in each class.

Interpretations. 
This shows that the nucleus stats, as captured by CP,
contain enough information to recapitulate the CNN attention.
This does not prove the CNN focused attention on the nuclei.
It is unclear why most (9 of top 10) important features
for the RF had to do with nucleus pixel coords within the patch;
elsewhere we demonstrated no correlation between CNN attention
and distance of the nucleus from the patch center.

Future work:
Redo the analysis for all classes, not per-class. 
Differentiate nuclei by whether their patch was correctly classified by the CNN. 
Rerun without the nucleus pixel coords, and with just the pixel coords.

## Attention.HalfRoll 1-5
This is a data preparation step only. 
As in Attention.1, we used average temperature per nucleus.
We wonder whether the HOT nuclei (those with high CNN attention)
are better predictors of cancer class than COLD nuclei.
Operating on each class separately, we generate 
statistical rollups of HOT nuclei per patch, and
statistical rollups of COLD nuclei per patch.
Note we already have 
statistical rollups of ALL nuclei per patch.





## Attention.Position.Correlation
Previously, we showed a RF could predict CNN attention per nucleus
with over 90% accuracy.
We were confused why most (9 of top 10) important features,
as ranked by the RF, had to do with nucleus location 
relative to the patch.
We speculated that attention prefers the center or the edges
of patches.
Here, we tested whether attention correlates to nucleus distance from the center of the patch.
There was zero correlation, leaving us with no explantion.

## Deciles
Look at distribution of nuclei per patch.
Nuclei were detected by CP.
Are there eoungh nuclei to support patch statistics
at the level of deciles (as opposed to, say, quartiles)?

## PatchNumbersUsedToTuneCellProfiler
A small number of patches were used to tune CellProfiler
parameters while building the Process100 pipeline.
Unfortunately, it turned out 4 were from our train set
and 4 were from our test set.
These patches should be excluded from validation and test statistics.

## RandomForest 20, 21, 22 
We tested whether HOT nuclei are better predictors
of cancer class than COLD nuclei.
This test indicated: no.

The HOT and COLD are based on the CNN attention heatmap.
A RF classifier was trained and evaluated by cross-validation.
Each used a quartile summary of nuclei per patch. 
Each has the same number of patches and features but the statistics 
like median nucleus size are based on different nuclei.
The HOT run used only high-attention nuclei 
(average temperature >= 50% within the nucleus bounding box.
The three notebooks used ALL, HOT, COLD respectively.
If HOT>COLD, it would say attention guides us to important nuclei.
If HOT>ALL, it would say attention guided us to nuclei so important
that using this half of the data is better than using all the data.
This notebook used classes 1-5 because
we didn't have attention results for class 0.

The results were HOT==COLD and HOT<ALL. 
This indicates the high-attention nuclei are not better
predictors than the low-attention nuclei.
We are confused how to reconcile this with other results.

## RandomForest 30, 31, 32, 33
So far, we used sklearn RandomForestClassifier with default parameters.
Should we try altering parameters?
These few tests indicated: no.

## RBC.Counts
We counted the RBC per class.
Some classes have more RBC than others.
Red blood cells tend to lie above and below the tissue plane
of the WSI.
We suspect high-RBC complicates patch classification
by obfuscating the nuclei during CellProfiler nucleus detection.
This suggests we should filter high-RBC patches
when making WSI-level predictions.

## WSI Stats
We only started thinking about aggregation
of patch-level predictions to the WSI level.